Test Details: We want to make and test functions which takes pdf and some questions and returns the top n pages we can get the answers on.

Author: Dhairya Kantawala

In [39]:
from langchain_community.document_loaders import PyPDFLoader
import pandas as pd
import os
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv("OPENAI_API")
from openai import OpenAI
client = OpenAI(api_key=api_key)
import numpy as np
from numpy.linalg import norm

In [35]:
def get_page_df(file_path):
    loader = PyPDFLoader(file_path)
    pages = loader.load()
    df = pd.DataFrame([i for i in range(len(pages))])
    for i in range(len(df)):
        df.loc[i, "page_content"] = pages[i].page_content
    df = df.drop(columns=0)
    return df

In [40]:
def get_embd(string_to_convert):
    response = client.embeddings.create(
        input=string_to_convert,
        model="text-embedding-3-small"
    )
    return response.data[0].embedding

In [43]:
def get_vector_df(filepath):
    df = get_page_df(filepath)
    df['embd'] = df['page_content'].apply(get_embd)
    return df

In [45]:
file_path = '/Users/dhairya/cs projects/SusReach/data/BRSR202324.pdf'
df = get_vector_df(file_path)

In [42]:
def cosine_similarity(first, second):
    cosine = np.dot(first, second) / (norm(first) * norm(second))
    return cosine

In [52]:
def page_number_on_embd_df(question_string, embd_df):
    max_index = 0
    max_score = 0
    question_embd = get_embd(question_string)
    for i in range(len(embd_df)):
        score = cosine_similarity(question_embd, embd_df['embd'].iloc[i])
        if score > max_score:
            max_score = score
            max_index = i
    return max_index

In [60]:
page_number_on_embd_df("""
CSR Details:
(i) Whether CSR is applicable as per section 135 of Companies Act, 2013: (Yes/No) 
(ii) Turnover (in Rs.) 
(iii) Net worth (in Rs.)"""
, df)

8